In [1]:
# %load ../startup.py
import os,sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ['PYTHONPATH']=os.environ.get('LIB')
if not os.environ.get('LIB') in sys.path: 
    sys.path.insert(0,os.environ.get('LIB'))

DAT_DIR = os.environ.get('DAT_DIR')    

%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
%load_ext rpy2.ipython

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 
from matplotlib import gridspec
from scipy.interpolate import splev, splrep
import xlwt
import seaborn as sns
from scipy import stats
import rpy2.robjects.packages as rpackages
import seaborn as sns
from statsmodels import robust


from heprnhci.db.mongo import *


from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>")) # increase jupyter screen width to 100%
pd.options.display.max_colwidth = 500
pd.set_option('display.precision',2)

HCI=openMongo(db=os.environ.get('MONGO_HCIDB'),host=os.environ.get('MONGO_HOST'),auth=False)


# Goal

Summarize and store in vitro assay results from 
* HepRn
* ToxCast & Tox21
* HepRnAtg


# Chemicals

In [1]:
from tk.httk import * 


RAT_LIV_LL = pd.read_excel(DAT_DIR+'toxref-v2.0-pods-rat-liver.xlsx')
CH_CAS0=pd.DataFrame(list(HCI.chemicals.find()),
                    columns=['dsstox_sid','chem_name','casrn'])\
        .rename(columns=dict(chem_name='name'))
CAS_TK=getChemInfo()
CAS0 = set(CH_CAS0.casrn).intersection(RAT_LIV_LL.casrn).intersection(CAS_TK)
CH_CAS1 = CH_CAS0[CH_CAS0.casrn.isin(CAS0)]

ModuleNotFoundError: No module named 'tk'

## HepRn

The HepRn CR analysis is conducted as follows:
* Curve fits are in notebooks/heprn-ivive/110-heprn-cr.ipynb and results stored in mongodb://localhost/heprn_v1.heprn_cr 
* Curve fit quality is analyzed in notebooks/heprn-ivive/130-heprn-cr-manual-qc.ipynb and stored in mongodb://localhost/heprn_v1/heprn_hitqual
* The final hits are identified based on quality and the results are stored in 
mongodb://localhost/heprn_v1/potency

### HepRn AC50 data

In [3]:
from box import Box
HEPRN = [Box(AC50=HepRn_filt1,potency_meth='tcpl:ac50'),
         Box(AC50=HepRn_qc,potency_meth='tcpl:ac50:manual_qc')]

NameError: name 'HepRn_filt1' is not defined

### Summarize HepRn data

In [ ]:
def _p05(X):  return np.percentile(X,5)
def _p95(X): return np.percentile(X,95)
def _p50(X): return np.percentile(X,50)
def _n(X): return len(X)


for D in HEPRN:
    D.AC50 = D.AC50.merge(CH_CAS0[['dsstox_sid','name']],on='name')
    
    HepRn = D.AC50.groupby(['dsstox_sid','name'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRn.columns=[i+j for i,j in HepRn.columns]
    HepRn = HepRn.reset_index()
    HepRn_L = HepRn.melt(['dsstox_sid','name'],
                         var_name='potency_name',value_name='potency_val')
    HepRn_L.insert(0,'assay','HepRn')
    HepRn_L.insert(0,'platform','APR:HepRn')
    HepRn_L.insert(0,'ft','All')
    HepRn_L.insert(0,'timeh',72)


    HepRnTm = D.AC50.groupby(['dsstox_sid','name','timeh'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRnTm.columns=[i+j for i,j in HepRnTm.columns]
    HepRnTm = HepRnTm.reset_index()


    HepRnTm_L = HepRnTm.melt(['dsstox_sid','name','timeh'],
                         var_name='potency_name',value_name='potency_val')
    HepRnTm_L.insert(0,'assay','HepRn')
    HepRnTm_L.insert(0,'platform','APR:HepRn:Tm')
    HepRnTm_L.insert(0,'ft','All')

    HepRnFtTm = D.AC50.groupby(['dsstox_sid','name','ft','timeh'])\
                .aggregate(dict(ac50=[_p05,_p50,_p95]))
    HepRnFtTm.columns=[i+j for i,j in HepRnFtTm.columns]
    HepRnFtTm = HepRnFtTm.reset_index()

    HepRnFtTm_L = HepRnFtTm.melt(['dsstox_sid','name','ft','timeh'],
                         var_name='potency_name',value_name='potency_val')
    HepRnFtTm_L.insert(0,'assay','HepRn')
    HepRnFtTm_L.insert(0,'platform','APR:HepRn:Tm:Ft')
    
    D.potency = pd.concat((HepRnTm_L,HepRn_L,HepRnFtTm_L))
    D.potency['days_ivt'] = D.potency.timeh/24
    D.potency.insert(D.potency.shape[1],'potency_meth',D.potency_meth)